In [27]:
!pip install langchain-teddynote

In [28]:
!pip install langchain_community

In [29]:
!pip install langchain_huggingface

In [32]:
import os

from google.colab import userdata

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "LangChain 실습"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('langsmith_api_key')

In [33]:
import zipfile

# 데이터 처리 및 분석
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt
import datetime

# 머신러닝 전처리
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import f1_score

# 모델 저장
import joblib

# To ignore all warnings
import warnings
warnings.filterwarnings('ignore')



In [34]:
from langchain_teddynote import logging

logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


In [35]:
from langchain_community.chat_models import ChatPerplexity
from langchain_core.prompts import ChatPromptTemplate

os.environ["PPLX_API_KEY"] = userdata.get('perplexity_api_key')


In [36]:
chat = ChatPerplexity(temperature=0,  model="llama-3.1-sonar-small-128k-chat")

In [37]:
prompt = ChatPromptTemplate.from_messages([
   # ("system", "You are a keyword finder, which specialized in finding exact HScode and product category from product's description."),
   # ("system", """You are a description summarizing tool, which specialized in summarizing certain product's description, which will be use to searcing appopriate overseas buyers based on their import history description.
   # Your answers have to be simple, and only contains 5 main keywords of description with order of importance. Your answers need to be one complete sentence, without any adjective or adverb. You do not need to explain why you choose this keywords. """),

    ("system", """Summarize product descriptions into 5 main keywords, ordered by importance. Output one sentence without adjectives or adverbs. No explanations."""),
    ("human", "{input}")
])

chain = prompt | chat

In [38]:
paragraph = """ASOME Intensive 7 Effect Total Cream – Visible reduction in dark spots in just 4 weeks.

A functional cream for whitening and wrinkle improvement, clinically proven for 7 key benefits.
Formulated for sensitive skin with a mild, vegan formulation to reduce skin irritation.

This lightweight cream absorbs quickly and glides smoothly onto the skin without any stickiness, helping to reveal a soft, refined skin texture.

With a delicate rose scent, it's suitable for all ages and is universally appealing."""

response = chain.invoke({"input": f"Extract 5 keywords identifying product category (e.g., cosmetics, snacks, heating appliances) from: {paragraph}"})
print(response.content)

Cosmetics, Skin Care, Whitening, Wrinkle Improvement, Sensitive Skin


In [41]:
from langchain_huggingface import HuggingFaceEmbeddings

# HuggingFaceEmbeddings 초기화
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# 임베딩할 텍스트
text = response

# 쿼리 텍스트 임베딩
query_embedding = embeddings.embed_query(text.content)

# 결과 출력 (처음 3개 값만)
print(query_embedding)

[0.049941536039114, 0.024973377585411072, -0.013476376421749592, -0.023682501167058945, 0.0005220131133683026, -0.0080494936555624, 0.08801785856485367, 0.09420328587293625, 0.05133524164557457, -0.012007679790258408, 0.025683568790555, 0.018574291840195656, 0.033603500574827194, 0.046628713607788086, 0.01971583068370819, 0.008030504919588566, -0.0063887727446854115, 0.04746392369270325, 0.020513631403446198, -0.036341339349746704, -0.02226172760128975, -0.008097672834992409, -0.022636402398347855, 0.06699351221323013, -0.004241478629410267, -0.0793185904622078, 0.005844963248819113, -0.00382618373259902, -0.016390135511755943, -0.02908296324312687, 0.03734813258051872, -0.01476171612739563, 0.0030401237308979034, -0.08791399747133255, 1.2743665820380556e-06, -0.011827943846583366, -0.03843628987669945, -0.027426278218626976, 0.001765978755429387, -0.003627181053161621, 0.09659034013748169, -0.031936269253492355, 0.013740519993007183, 0.005519242025911808, 0.0025458578020334244, 0.0064